# 引用包

In [1]:
# encoding=utf-8
import os.path as osp
import os
import copy
import matplotlib.pyplot as plt
import torch
from torch.nn import Linear
from sklearn.metrics import average_precision_score, roc_auc_score
from torch_geometric.data import TemporalData
from torch_geometric.datasets import JODIEDataset
from torch_geometric.datasets import ICEWS18
from torch_geometric.nn import TGNMemory, TransformerConv
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn.models.tgn import (LastNeighborLoader, IdentityMessage, MeanAggregator,
                                           LastAggregator)
from torch_geometric import *
from torch_geometric.utils import negative_sampling
from tqdm import tqdm
import networkx as nx
import numpy as np
import math
import copy
import re
import time
import json
import pandas as pd
from random import choice
import gc
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
# msg的特征采取    [src_node_feature,edge_attr,dst_node_feature]的格式

# compute the best partition 计算出最佳的社区划分
import datetime
# import community as community_louvain

# import xxhash
# 查找edge向量所对应的下标
def tensor_find(t,x):
    t_np=t.cpu().numpy()
    idx=np.argwhere(t_np==x)
    return idx[0][0]+1


def std(t):
    t = np.array(t)
    return np.std(t)


def var(t):
    t = np.array(t)
    return np.var(t)


def mean(t):
    t = np.array(t)
    return np.mean(t)

def hashgen(l):
    """Generate a single hash value from a list. @l is a list of
    string values, which can be properties of a node/edge. This
    function returns a single hashed integer value."""
    hasher = xxhash.xxh64()
    for e in l:
        hasher.update(e)
    return hasher.intdigest()

# 单独计算出每条边的loss值来分析出 具体的异常行为
def cal_pos_edges_loss(link_pred_ratio):
    loss=[]
    for i in link_pred_ratio:
        loss.append(criterion(i,torch.ones(1)))
    return torch.tensor(loss)

def cal_pos_edges_loss_multiclass(link_pred_ratio,labels):
    loss=[] 
    for i in range(len(link_pred_ratio)):
        loss.append(criterion(link_pred_ratio[i].reshape(1,-1),labels[i].reshape(-1)))
    return torch.tensor(loss)

def cal_pos_edges_loss_autoencoder(decoded,msg):
    loss=[] 
    for i in range(len(decoded)):
        loss.append(criterion(decoded[i].reshape(1,-1),msg[i].reshape(-1)))
    return torch.tensor(loss)

In [2]:
%autosave 120  
# 0为取消自动保存

Autosaving every 120 seconds


In [3]:
# 时间戳转换函数

from datetime import datetime, timezone
import time
import pytz
from time import mktime
from datetime import datetime
import time
def ns_time_to_datetime(ns):
    """
    :param ns: int 纳秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00.000000000
    """
    dt = datetime.fromtimestamp(int(ns) // 1000000000)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s


def ns_time_to_datetime_US(ns):
    """
    :param ns: int 纳秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00.000000000
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(ns) // 1000000000, tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s

def time_to_datetime_US(s):
    """
    :param ns: int 秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(s), tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    
    return s

def datetime_to_ns_time(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    timeStamp = int(time.mktime(timeArray))
    timeStamp = timeStamp * 1000000000
    return timeStamp

def datetime_to_ns_time_US(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp=timestamp.timestamp()
    timeStamp = timestamp * 1000000000
    return int(timeStamp)

def datetime_to_timestamp_US(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp=timestamp.timestamp()
    timeStamp = timestamp
    return int(timeStamp)


## 连接数据库

In [4]:
# 连接postgessql 数据库，将数据存入数据库中
import psycopg2

from psycopg2 import extras as ex
connect = psycopg2.connect(database = 'tc_cadet_dataset_db',
                           host = '/var/run/postgresql/',
                           user = 'postgres',
                           password = 'postgres',
                           port = '5432'#一般是5432
                          )



# 创建一个cursor来执行数据库的操作
cur = connect.cursor()

## 加载数据

In [6]:
graph_4_2=torch.load("./train_graph/graph_4_2.TemporalData.simple-share").to(device=device)
graph_4_3=torch.load("./train_graph/graph_4_3.TemporalData.simple-share").to(device=device)
graph_4_4=torch.load("./train_graph/graph_4_4.TemporalData.simple-share").to(device=device)
graph_4_5=torch.load("./train_graph/graph_4_5.TemporalData.simple-share").to(device=device)
graph_4_6=torch.load("./train_graph/graph_4_6.TemporalData.simple-share").to(device=device)
graph_4_7=torch.load("./train_graph/graph_4_7.TemporalData.simple-share").to(device=device)

train_data=graph_4_2

In [7]:
normal_count=0
for g in graph_list:
    normal_count+=len(g.t)

NameError: name 'graph_list' is not defined

In [8]:
normal_count

0

## 生成node2msg

In [9]:
# 构建nodeid 与msg之间的构建  创建字典变量 nodeid2msg
sql="select * from node2id ORDER BY index_id;"
cur.execute(sql)
rows = cur.fetchall()

nodeid2msg={}  # 可以实现 nodeid 转 msg      node hash 转 nodeid
for i in rows:
    nodeid2msg[i[0]]=i[-1]
    nodeid2msg[i[-1]]={i[1]:i[2]}  

In [10]:
rel2id={1: 'EVENT_WRITE',
 'EVENT_WRITE': 1,
 2: 'EVENT_READ',
 'EVENT_READ': 2,
 3: 'EVENT_CLOSE',
 'EVENT_CLOSE': 3,
 4: 'EVENT_OPEN',
 'EVENT_OPEN': 4,
 5: 'EVENT_EXECUTE',
 'EVENT_EXECUTE': 5,
 6: 'EVENT_SENDTO',
 'EVENT_SENDTO': 6,
 7: 'EVENT_RECVFROM',
 'EVENT_RECVFROM': 7}

# 配置 设置 模型

## one hop 邻居

In [15]:
graph_4_3.num_nodes,graph_4_4.num_nodes,graph_4_5.num_nodes,graph_4_6.num_nodes,graph_4_7.num_nodes,

(201716, 205538, 210843, 224653, 240160)

In [11]:
# train_data, val_data, test_data = data.train_val_test_split(val_ratio=0.15, test_ratio=0.15)
# max_node_num = max(torch.cat([data.dst,data.src]))+1
# max_node_num = data.num_nodes+1
max_node_num = 268243  # +1
# min_dst_idx, max_dst_idx = int(data.dst.min()), int(data.dst.max())
min_dst_idx, max_dst_idx = 0, max_node_num
neighbor_loader = LastNeighborLoader(max_node_num, size=20, device=device)

## 多分类重建edge type训练方式

In [12]:
class GraphAttentionEmbedding(torch.nn.Module):
    def __init__(self, in_channels, out_channels, msg_dim, time_enc):
        super(GraphAttentionEmbedding, self).__init__()
        self.time_enc = time_enc
        edge_dim = msg_dim + time_enc.out_channels
        self.conv = TransformerConv(in_channels, out_channels, heads=8,
                                    dropout=0.0, edge_dim=edge_dim)
        self.conv2 = TransformerConv(out_channels*8, out_channels,heads=1, concat=False,
                             dropout=0.0, edge_dim=edge_dim)

    def forward(self, x, last_update, edge_index, t, msg):
        last_update.to(device)
        x = x.to(device)
        t = t.to(device)
        rel_t = last_update[edge_index[0]] - t
        rel_t_enc = self.time_enc(rel_t.to(x.dtype))
        edge_attr = torch.cat([rel_t_enc, msg], dim=-1)
        x = F.relu(self.conv(x, edge_index, edge_attr))
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super(LinkPredictor, self).__init__()
        self.lin_src = Linear(in_channels, in_channels*2)
        self.lin_dst = Linear(in_channels, in_channels*2)
        
        self.lin_seq = nn.Sequential(
            
            Linear(in_channels*4, in_channels*8),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(in_channels*8, in_channels*2),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(in_channels*2, int(in_channels//2)),
            torch.nn.Dropout(0.5),
            nn.Tanh(),
            Linear(int(in_channels//2), train_data.msg.shape[1]-32)                   
        )
        

    def forward(self, z_src, z_dst):
        h = torch.cat([self.lin_src(z_src) , self.lin_dst(z_dst)],dim=-1)      
         
        h = self.lin_seq (h)
        
        return h

memory_dim = time_dim = embedding_dim = 100

memory = TGNMemory(
    max_node_num,
    train_data.msg.size(-1),
    memory_dim,
    time_dim,
    message_module=IdentityMessage(train_data.msg.size(-1), memory_dim, time_dim),
    aggregator_module=LastAggregator(),
).to(device)

gnn = GraphAttentionEmbedding(
    in_channels=memory_dim,
    out_channels=embedding_dim,
    msg_dim=train_data.msg.size(-1),
    time_enc=memory.time_enc,
).to(device)

link_pred = LinkPredictor(in_channels=embedding_dim).to(device)

optimizer = torch.optim.Adam(
    set(memory.parameters()) | set(gnn.parameters())
    | set(link_pred.parameters()), lr=0.00005, eps=1e-08,weight_decay=0.01)


# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.CrossEntropyLoss()

# Helper vector to map global node indices to local ones.
assoc = torch.empty(max_node_num, dtype=torch.long, device=device)

saved_nodes=set()

In [13]:
link_pred

LinkPredictor(
  (lin_src): Linear(in_features=100, out_features=200, bias=True)
  (lin_dst): Linear(in_features=100, out_features=200, bias=True)
  (lin_seq): Sequential(
    (0): Linear(in_features=400, out_features=800, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): Tanh()
    (3): Linear(in_features=800, out_features=200, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): Tanh()
    (6): Linear(in_features=200, out_features=50, bias=True)
    (7): Dropout(p=0.5, inplace=False)
    (8): Tanh()
    (9): Linear(in_features=50, out_features=7, bias=True)
  )
)

In [14]:
BATCH=1024
def train(train_data):

    
    memory.train()
    gnn.train()
    link_pred.train()

    memory.reset_state()  # Start with a fresh memory.
    neighbor_loader.reset_state()  # Start with an empty graph.
    saved_nodes=set()

    total_loss = 0
    
#     print("train_before_stage_data:",train_data)
    for batch in train_data.seq_batches(batch_size=BATCH):
        optimizer.zero_grad()

        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg        
        
        n_id = torch.cat([src, pos_dst]).unique()
#         n_id = torch.cat([src, pos_dst, neg_src, neg_dst]).unique()
        n_id, edge_index, e_id = neighbor_loader(n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        # Get updated memory of all nodes involved in the computation.
        z, last_update = memory(n_id)
      
        z = gnn(z, last_update, edge_index, train_data.t[e_id], train_data.msg[e_id])
        
        pos_out = link_pred(z[assoc[src]], z[assoc[pos_dst]])       

        y_pred = torch.cat([pos_out], dim=0)
        
#         y_true = torch.cat([torch.zeros(pos_out.size(0),1),torch.ones(neg_out.size(0),1)], dim=0)# 0 代表正常 1 代表异常
        y_true=[]
        for m in msg:
            l=tensor_find(m[16:-16],1)-1
            y_true.append(l)           
          
        y_true = torch.tensor(y_true).to(device=device)
        y_true=y_true.reshape(-1).to(torch.long).to(device=device)
        
        loss = criterion(y_pred, y_true)
        
# 原先代码中的loss计算方法
#         loss = criterion(pos_out, torch.ones_like(pos_out))
#         loss += criterion(neg_out, torch.zeros_like(neg_out))

        # Update memory and neighbor loader with ground-truth state.
        memory.update_state(src, pos_dst, t, msg)
        neighbor_loader.insert(src, pos_dst)
        
#         for i in range(len(src)):
#             saved_nodes.add(int(src[i]))
#             saved_nodes.add(int(pos_dst[i]))

        loss.backward()
        optimizer.step()
        memory.detach()
#         print(z.shape)
        total_loss += float(loss) * batch.num_events
#     print("trained_stage_data:",train_data)
    return total_loss / train_data.num_events


# 开始训练

In [16]:
# 4.3  4.4 可以用于训练  
train_graphs=[graph_4_2,graph_4_3,graph_4_4]

for epoch in tqdm(range(1, 51)):
    for g in train_graphs:
        loss = train(g)
        print(f'  Epoch: {epoch:02d}, Loss: {loss:.4f}')
#     scheduler.step()
    # 将训练好的模型保存下来
model=[memory,gnn, link_pred,neighbor_loader]
torch.save(model,"./models/model_saved_share.pt")

  0%|                                                                                                   | 0/50 [00:00<?, ?it/s]

  Epoch: 01, Loss: 1.2058
  Epoch: 01, Loss: 0.9020


  2%|█▊                                                                                         | 1/50 [01:06<54:28, 66.71s/it]

  Epoch: 01, Loss: 0.7853
  Epoch: 02, Loss: 0.6923
  Epoch: 02, Loss: 0.7264


  4%|███▋                                                                                       | 2/50 [02:12<53:09, 66.45s/it]

...

  Epoch: 48, Loss: 0.6330
  Epoch: 49, Loss: 0.5845
  Epoch: 49, Loss: 0.6320


 98%|████████████████████████████████████████████████████████████████████████████████████████▏ | 49/50 [54:38<01:07, 67.03s/it]

  Epoch: 49, Loss: 0.6328
  Epoch: 50, Loss: 0.5846
  Epoch: 50, Loss: 0.6292


100%|██████████████████████████████████████████████████████████████████████████████████████████| 50/50 [55:44<00:00, 66.89s/it]

  Epoch: 50, Loss: 0.6312


## 测试使用的函数

In [17]:
import time 

@torch.no_grad()#声明以下函数不执行梯度
def test_day_new(inference_data,path):
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)
    
    memory.eval()
    gnn.eval()
    link_pred.eval()
    
    memory.reset_state()  # Start with a fresh memory.  # 为什么不可以使用历史的memory呢？  应该 可以吧？
    neighbor_loader.reset_state()  # Start with an empty graph.
    
    time_with_loss={}# key: 时间段，  value： 该时间段的loss值
    total_loss = 0    
    edge_list=[]
    
    unique_nodes=torch.tensor([]).to(device=device)
    total_edges=0

# 记录起始的时间点

    start_time=inference_data.t[0]
    event_count=0
    
    pos_o=[]
    
    loss_list=[]
    

    print("after merge:",inference_data)
    
    # 记录程序运行时间  评估运行效率
    start = time.perf_counter()

    for batch in inference_data.seq_batches(batch_size=BATCH):
        
        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg
        unique_nodes=torch.cat([unique_nodes,src,pos_dst]).unique()
        total_edges+=BATCH
        
       
        n_id = torch.cat([src, pos_dst]).unique()       
        n_id, edge_index, e_id = neighbor_loader(n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        z, last_update = memory(n_id)
        #如果memory和neighbor_loader在测试阶段都没有被更新的话，此处不需要采用gnn获得z矩阵，只需要从memory查出来对应的节点特征向量即可
        z = gnn(z, last_update, edge_index, inference_data.t[e_id], inference_data.msg[e_id])

        pos_out = link_pred(z[assoc[src]], z[assoc[pos_dst]])
        
        pos_o.append(pos_out)
        y_pred = torch.cat([pos_out], dim=0)
#         y_true = torch.cat(
#             [torch.ones(pos_out.size(0))], dim=0).to(torch.long)     
#         y_true=y_true.reshape(-1).to(torch.long)

        y_true=[]
        for m in msg:
            l=tensor_find(m[16:-16],1)-1
            y_true.append(l) 
        y_true = torch.tensor(y_true).to(device=device)
        y_true=y_true.reshape(-1).to(torch.long).to(device=device)

        # 只考虑边有没有被正确预测，对于正常行为的图而言，行为模式比较相似所以loss较低。  对于异常行为，会存在一些行为没见过，所以对这些行为预测存在的概率就地，所以loss也会高。
        loss = criterion(y_pred, y_true)

        total_loss += float(loss) * batch.num_events
     
        
# 将当前batch 发生的edge 更新到memory 和neighbor中
        memory.update_state(src, pos_dst, t, msg)
        neighbor_loader.insert(src, pos_dst)
        
        #计算每条边的loss值
        each_edge_loss= cal_pos_edges_loss_multiclass(pos_out,y_true)
        
        for i in range(len(pos_out)):
            srcnode=int(src[i])
            dstnode=int(pos_dst[i])  
            
            srcmsg=str(nodeid2msg[srcnode]) 
            dstmsg=str(nodeid2msg[dstnode])
            t_var=int(t[i])
            edgeindex=tensor_find(msg[i][16:-16],1) # find 找出来的范围是 1-n   rel2id中的id也是1-n    
            edge_type=rel2id[edgeindex]
            loss=each_edge_loss[i]    

            temp_dic={}
            temp_dic['loss']=float(loss)
            temp_dic['srcnode']=srcnode
            temp_dic['dstnode']=dstnode
            temp_dic['srcmsg']=srcmsg
            temp_dic['dstmsg']=dstmsg
            temp_dic['edge_type']=edge_type
            temp_dic['time']=t_var
            
            # 先不考虑与socket 节点 当找出可疑的进程与文件之后再将socket找出
#             if "netflow" in srcmsg or "netflow" in dstmsg:
#                 temp_dic['loss']=0
            edge_list.append(temp_dic)
        
        event_count+=len(batch.src)
        if t[-1]>start_time+60000000000*15:
            # 此处为一个checkpoint  输出loss值 并且清空全局的loss值  保存处理过的节点
#             loss=total_loss/event_count
            time_interval=ns_time_to_datetime_US(start_time)+"~"+ns_time_to_datetime_US(t[-1])

            end = time.perf_counter()
            time_with_loss[time_interval]={'loss':loss,
                                
                                          'nodes_count':len(unique_nodes),
                                          'total_edges':total_edges,
                                          'costed_time':(end-start)}
            
            
            log=open(path+"/"+time_interval+".txt",'w')
            # 减去train data中没有被训练好的
            
            for e in edge_list: 
#                 temp_key=e['srcmsg']+e['dstmsg']+e['edge_type']
#                 if temp_key in train_edge_set:      
# #                     e['loss']=(e['loss']-train_edge_set[temp_key]) if e['loss']>=train_edge_set[temp_key] else 0  
# #                     e['loss']=abs(e['loss']-train_edge_set[temp_key])
                    
#                     e['modified']=True
#                 else:
#                     e['modified']=False
                loss+=e['loss']

            loss=loss/event_count   
            print(f'Time: {time_interval}, Loss: {loss:.4f}, Nodes_count: {len(unique_nodes)}, Cost Time: {(end-start):.2f}s')
            edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True)   # 按照loss 值排序  或者按照edge的时间顺序排列
            for e in edge_list: 
                log.write(str(e))
                log.write("\n") 
            event_count=0
            total_loss=0
            loss=0
            start_time=t[-1]
            log.close()
            edge_list.clear()
            
 
    return time_with_loss


# 测试环节

## 按照每天的数据进行测试

In [18]:
def status(x) : 
     return pd.Series([x.count(),x.min(),x.idxmin(),x.quantile(.25),x.median(),
       x.quantile(.75),x.mean(),x.max(),x.idxmax(),x.mad(),x.var(),
       x.std(),x.skew(),x.kurt()],index=['总数','最小值','最小值位置','25%分位数',
       '中位数','75%分位数','均值','最大值','最大值位数','平均绝对偏差','方差','标准差','偏度','峰度'])

## 生成label

In [28]:
labels={}
filelist = os.listdir("graph_4_6_share")
for f in filelist:
    labels[f]=0
filelist = os.listdir("graph_4_7_share")
for f in filelist:
    labels[f]=0   
    

In [29]:
attack_list=[
    '2018-04-06 11:18:26.126177915~2018-04-06 11:33:35.116170745.txt',
    '2018-04-06 11:33:35.116170745~2018-04-06 11:48:42.606135188.txt',
    '2018-04-06 11:48:42.606135188~2018-04-06 12:03:50.186115455.txt',
    '2018-04-06 12:03:50.186115455~2018-04-06 14:01:32.489584227.txt',
]
for i in attack_list:
    labels[i]=1

    

In [30]:
labels

{'2018-04-06 14:16:39.379560570~2018-04-06 14:31:41.149538564.txt': 0,
 '2018-04-06 18:40:52.729207049~2018-04-06 18:55:59.999193978.txt': 0,
 '2018-04-06 09:16:25.066340315~2018-04-06 09:31:25.146320510.txt': 0,
 '2018-04-06 02:33:48.406880632~2018-04-06 02:49:07.976859935.txt': 0,
 '2018-04-06 23:03:09.188854937~2018-04-06 23:18:10.168840981.txt': 0,
 '2018-04-06 22:01:32.928937945~2018-04-06 22:17:39.688914711.txt': 0,
 '2018-04-06 00:00:00.017083676~2018-04-06 00:15:50.177068871.txt': 0,
 '2018-04-06 07:28:29.926483734~2018-04-06 07:43:36.186466909.txt': 0,
 '2018-04-06 00:30:58.967045191~2018-04-06 00:47:07.167023337.txt': 0,
 '2018-04-06 17:20:51.639321683~2018-04-06 17:35:54.449302990.txt': 0,
 '2018-04-06 04:22:56.346731444~2018-04-06 04:38:03.996711152.txt': 0,
 '2018-04-06 04:07:44.916766067~2018-04-06 04:22:56.346731444.txt': 0,
 '2018-04-06 23:33:15.408814807~2018-04-06 23:49:23.948792776.txt': 0,
 '2018-04-06 01:47:34.436947077~2018-04-06 02:02:43.106920556.txt': 0,
 '2018

## 统计与攻击相关的edge数量

In [31]:
def keyword_hit(line):
    attack_nodes=[
            'vUgefal',        
        '/var/log/devc',
        'nginx',   
        '81.49.200.166',
      '78.205.235.65',
        '200.36.109.214',
        '139.123.0.113',
        '152.111.159.139',
        '61.167.39.128',
             
        ]
    flag=False
    for i in attack_nodes:
        if i in line:
            flag=True
            break
    return flag

files=[]
temp_file=[
   '2018-04-06 11:18:26.126177915~2018-04-06 11:33:35.116170745.txt',
    '2018-04-06 11:33:35.116170745~2018-04-06 11:48:42.606135188.txt',
    '2018-04-06 11:48:42.606135188~2018-04-06 12:03:50.186115455.txt',
    '2018-04-06 12:03:50.186115455~2018-04-06 14:01:32.489584227.txt',
]
for f in temp_file:
    files.append("./graph_4_6_share/"+f)
    

In [32]:
files

['./graph_4_6_share/2018-04-06 11:18:26.126177915~2018-04-06 11:33:35.116170745.txt',
 './graph_4_6_share/2018-04-06 11:33:35.116170745~2018-04-06 11:48:42.606135188.txt',
 './graph_4_6_share/2018-04-06 11:48:42.606135188~2018-04-06 12:03:50.186115455.txt',
 './graph_4_6_share/2018-04-06 12:03:50.186115455~2018-04-06 14:01:32.489584227.txt']

In [33]:
attack_edge_count=0
for fpath in (files):
    f=open(fpath)
    for line in f:
        if keyword_hit(line):
            attack_edge_count+=1
print(attack_edge_count)

1248


## 测试  4-6\~7

In [16]:
# 模型训练完以后执行下面这些测试函数，获取model对测试数据的重构误差输出

In [22]:
model=torch.load("./models/model_saved_share.pt",map_location=device)
memory,gnn, link_pred,neighbor_loader=model
ans_4_2=test_day_new(graph_4_2,"graph_4_2_share")

after merge: TemporalData(dst=[445378], msg=[445378, 39], src=[445378], t=[445378])
Time: 2018-04-02 18:07:41.933350905~2018-04-02 18:23:52.893332321, Loss: 0.5053, Nodes_count: 154, Cost Time: 0.95s
Time: 2018-04-02 18:23:52.893332321~2018-04-02 18:39:05.813316237, Loss: 0.6493, Nodes_count: 171, Cost Time: 1.55s
Time: 2018-04-02 18:39:05.813316237~2018-04-02 18:55:00.023294921, Loss: 0.6296, Nodes_count: 171, Cost Time: 2.14s
Time: 2018-04-02 18:55:00.023294921~2018-04-02 19:10:00.033270003, Loss: 0.6499, Nodes_count: 181, Cost Time: 2.73s
Time: 2018-04-02 19:10:00.033270003~2018-04-02 19:25:30.803248092, Loss: 0.4138, Nodes_count: 183, Cost Time: 3.65s
Time: 2018-04-02 19:25:30.803248092~2018-04-02 19:40:44.033225802, Loss: 0.6369, Nodes_count: 183, Cost Time: 4.26s
Time: 2018-04-02 19:40:44.033225802~2018-04-02 19:55:53.713212284, Loss: 0.6438, Nodes_count: 183, Cost Time: 4.85s
Time: 2018-04-02 19:55:53.713212284~2018-04-02 20:11:03.013189297, Loss: 0.7162, Nodes_count: 209, Cost 

In [23]:
model=torch.load("./models/model_saved_share.pt",map_location=device)
memory,gnn, link_pred,neighbor_loader=model
ans_4_3=test_day_new(graph_4_3,"graph_4_3_share")

after merge: TemporalData(dst=[1758192], msg=[1758192, 39], src=[1758192], t=[1758192])
Time: 2018-04-03 00:00:00.002884014~2018-04-03 00:15:29.682868556, Loss: 0.7358, Nodes_count: 15488, Cost Time: 6.66s
Time: 2018-04-03 00:15:29.682868556~2018-04-03 00:30:39.652840273, Loss: 0.4029, Nodes_count: 15501, Cost Time: 7.85s
Time: 2018-04-03 00:30:39.652840273~2018-04-03 00:45:48.612820751, Loss: 0.6497, Nodes_count: 15501, Cost Time: 8.45s
Time: 2018-04-03 00:45:48.612820751~2018-04-03 01:00:57.722800088, Loss: 0.6538, Nodes_count: 15501, Cost Time: 9.11s
Time: 2018-04-03 01:00:57.722800088~2018-04-03 01:17:06.072775830, Loss: 0.6521, Nodes_count: 15515, Cost Time: 9.70s
Time: 2018-04-03 01:17:06.072775830~2018-04-03 01:32:16.902757103, Loss: 0.4067, Nodes_count: 15516, Cost Time: 10.62s
Time: 2018-04-03 01:32:16.902757103~2018-04-03 01:47:26.372745724, Loss: 0.6390, Nodes_count: 15516, Cost Time: 11.22s
Time: 2018-04-03 01:47:26.372745724~2018-04-03 02:02:35.472713620, Loss: 0.6504, Nod

Time: 2018-04-03 17:51:43.161442042~2018-04-03 18:07:48.711422162, Loss: 0.8387, Nodes_count: 18386, Cost Time: 66.94s
Time: 2018-04-03 18:07:48.711422162~2018-04-03 18:23:56.701398779, Loss: 0.6349, Nodes_count: 18411, Cost Time: 68.28s
Time: 2018-04-03 18:23:56.701398779~2018-04-03 18:40:05.281378008, Loss: 0.6717, Nodes_count: 18413, Cost Time: 68.96s
Time: 2018-04-03 18:40:05.281378008~2018-04-03 18:55:13.631357905, Loss: 0.6528, Nodes_count: 18413, Cost Time: 69.55s
Time: 2018-04-03 18:55:13.631357905~2018-04-03 19:11:19.271336540, Loss: 0.6733, Nodes_count: 18415, Cost Time: 70.20s
Time: 2018-04-03 19:11:19.271336540~2018-04-03 19:27:28.951316637, Loss: 0.4407, Nodes_count: 18418, Cost Time: 71.18s
Time: 2018-04-03 19:27:28.951316637~2018-04-03 19:43:37.151290672, Loss: 0.6598, Nodes_count: 18420, Cost Time: 71.85s
Time: 2018-04-03 19:43:37.151290672~2018-04-03 19:58:44.441279824, Loss: 0.6484, Nodes_count: 18420, Cost Time: 72.44s
Time: 2018-04-03 19:58:44.441279824~2018-04-03 2

In [24]:
model=torch.load("./models/model_saved_share.pt",map_location=device)
memory,gnn, link_pred,neighbor_loader=model
ans_4_4=test_day_new(graph_4_4,"graph_4_4_share")

after merge: TemporalData(dst=[1677700], msg=[1677700, 39], src=[1677700], t=[1677700])
Time: 2018-04-04 00:00:00.010948818~2018-04-04 00:15:00.870926016, Loss: 0.7506, Nodes_count: 118, Cost Time: 0.62s
Time: 2018-04-04 00:15:00.870926016~2018-04-04 00:30:58.170905986, Loss: 0.4211, Nodes_count: 133, Cost Time: 1.54s
Time: 2018-04-04 00:30:58.170905986~2018-04-04 00:46:06.990886590, Loss: 0.6696, Nodes_count: 150, Cost Time: 2.20s
Time: 2018-04-04 00:46:06.990886590~2018-04-04 01:02:14.890863698, Loss: 0.6861, Nodes_count: 152, Cost Time: 2.85s
Time: 2018-04-04 01:02:14.890863698~2018-04-04 01:18:19.710844541, Loss: 0.6377, Nodes_count: 152, Cost Time: 3.44s
Time: 2018-04-04 01:18:19.710844541~2018-04-04 01:33:30.220824602, Loss: 0.4396, Nodes_count: 155, Cost Time: 4.43s
Time: 2018-04-04 01:33:30.220824602~2018-04-04 01:48:37.810800940, Loss: 0.6759, Nodes_count: 157, Cost Time: 5.04s
Time: 2018-04-04 01:48:37.810800940~2018-04-04 02:03:45.210785278, Loss: 0.6626, Nodes_count: 157, C

Time: 2018-04-04 18:01:39.579497690~2018-04-04 18:16:41.079478593, Loss: 0.6861, Nodes_count: 1176, Cost Time: 63.02s
Time: 2018-04-04 18:16:41.079478593~2018-04-04 18:31:55.609456605, Loss: 0.4609, Nodes_count: 1180, Cost Time: 64.07s
Time: 2018-04-04 18:31:55.609456605~2018-04-04 18:47:01.579438295, Loss: 0.6727, Nodes_count: 1181, Cost Time: 64.74s
Time: 2018-04-04 18:47:01.579438295~2018-04-04 19:02:11.359417321, Loss: 0.6837, Nodes_count: 1183, Cost Time: 65.45s
Time: 2018-04-04 19:02:11.359417321~2018-04-04 19:18:19.169395880, Loss: 0.7077, Nodes_count: 1187, Cost Time: 66.16s
Time: 2018-04-04 19:18:19.169395880~2018-04-04 19:34:27.549373985, Loss: 0.4613, Nodes_count: 1190, Cost Time: 67.21s
Time: 2018-04-04 19:34:27.549373985~2018-04-04 19:49:35.319353447, Loss: 0.6641, Nodes_count: 1191, Cost Time: 67.88s
Time: 2018-04-04 19:49:35.319353447~2018-04-04 20:05:43.199330783, Loss: 0.7041, Nodes_count: 1209, Cost Time: 68.59s
Time: 2018-04-04 20:05:43.199330783~2018-04-04 20:20:50.

In [25]:
model=torch.load("./models/model_saved_share.pt",map_location=device)
memory,gnn, link_pred,neighbor_loader=model
ans_4_5=test_day_new(graph_4_5,"graph_4_5_share")

after merge: TemporalData(dst=[1936142], msg=[1936142, 39], src=[1936142], t=[1936142])
Time: 2018-04-05 00:00:00.019021794~2018-04-05 00:15:00.828996051, Loss: 0.7565, Nodes_count: 131, Cost Time: 0.70s
Time: 2018-04-05 00:15:00.828996051~2018-04-05 00:31:53.958984540, Loss: 0.4676, Nodes_count: 145, Cost Time: 1.77s
Time: 2018-04-05 00:31:53.958984540~2018-04-05 00:47:02.738955442, Loss: 0.6852, Nodes_count: 148, Cost Time: 2.52s
Time: 2018-04-05 00:47:02.738955442~2018-04-05 01:02:10.248933422, Loss: 0.6729, Nodes_count: 148, Cost Time: 3.21s
Time: 2018-04-05 01:02:10.248933422~2018-04-05 01:17:16.198913636, Loss: 0.7026, Nodes_count: 151, Cost Time: 3.89s
Time: 2018-04-05 01:17:16.198913636~2018-04-05 01:32:24.868893768, Loss: 0.4272, Nodes_count: 152, Cost Time: 4.91s
Time: 2018-04-05 01:32:24.868893768~2018-04-05 01:48:31.268873586, Loss: 0.7066, Nodes_count: 170, Cost Time: 5.67s
Time: 2018-04-05 01:48:31.268873586~2018-04-05 02:03:38.918851533, Loss: 0.6718, Nodes_count: 170, C

Time: 2018-04-05 18:00:00.897576901~2018-04-05 18:15:51.217550065, Loss: 0.7153, Nodes_count: 1992, Cost Time: 75.00s
Time: 2018-04-05 18:15:51.217550065~2018-04-05 18:31:58.647526739, Loss: 0.5169, Nodes_count: 2005, Cost Time: 76.21s
Time: 2018-04-05 18:31:58.647526739~2018-04-05 18:47:07.857504183, Loss: 0.7276, Nodes_count: 2017, Cost Time: 77.09s
Time: 2018-04-05 18:47:07.857504183~2018-04-05 19:02:51.597488187, Loss: 0.7451, Nodes_count: 2029, Cost Time: 77.96s
Time: 2018-04-05 19:02:51.597488187~2018-04-05 19:19:21.927461058, Loss: 0.7207, Nodes_count: 2038, Cost Time: 78.83s
Time: 2018-04-05 19:19:21.927461058~2018-04-05 19:34:28.807441594, Loss: 0.5881, Nodes_count: 2057, Cost Time: 80.16s
Time: 2018-04-05 19:34:28.807441594~2018-04-05 19:50:39.457421452, Loss: 0.7469, Nodes_count: 2068, Cost Time: 81.11s
Time: 2018-04-05 19:50:39.457421452~2018-04-05 20:05:43.727406866, Loss: 0.9598, Nodes_count: 2133, Cost Time: 82.33s
Time: 2018-04-05 20:05:43.727406866~2018-04-05 20:20:53.

In [26]:
model=torch.load("./models/model_saved_share.pt",map_location=device)
memory,gnn, link_pred,neighbor_loader=model
ans_4_6=test_day_new(graph_4_6,"graph_4_6_share")

after merge: TemporalData(dst=[2022481], msg=[2022481, 39], src=[2022481], t=[2022481])
Time: 2018-04-06 00:00:00.017083676~2018-04-06 00:15:50.177068871, Loss: 0.7886, Nodes_count: 166, Cost Time: 0.81s
Time: 2018-04-06 00:15:50.177068871~2018-04-06 00:30:58.967045191, Loss: 0.5329, Nodes_count: 201, Cost Time: 2.02s
Time: 2018-04-06 00:30:58.967045191~2018-04-06 00:47:07.167023337, Loss: 0.7404, Nodes_count: 217, Cost Time: 2.96s
Time: 2018-04-06 00:47:07.167023337~2018-04-06 01:02:14.677010991, Loss: 0.7295, Nodes_count: 231, Cost Time: 3.82s
Time: 2018-04-06 01:02:14.677010991~2018-04-06 01:17:19.056982719, Loss: 0.7198, Nodes_count: 240, Cost Time: 4.57s
Time: 2018-04-06 01:17:19.056982719~2018-04-06 01:32:29.456961759, Loss: 0.4836, Nodes_count: 248, Cost Time: 5.70s
Time: 2018-04-06 01:32:29.456961759~2018-04-06 01:47:34.436947077, Loss: 0.7561, Nodes_count: 264, Cost Time: 6.58s
Time: 2018-04-06 01:47:34.436947077~2018-04-06 02:02:43.106920556, Loss: 0.7210, Nodes_count: 276, C

Time: 2018-04-06 19:41:23.549124701~2018-04-06 19:57:28.259104051, Loss: 0.7964, Nodes_count: 6337, Cost Time: 86.49s
Time: 2018-04-06 19:57:28.259104051~2018-04-06 20:13:39.769084097, Loss: 0.9814, Nodes_count: 6374, Cost Time: 87.65s
Time: 2018-04-06 20:13:39.769084097~2018-04-06 20:28:47.019060830, Loss: 0.6807, Nodes_count: 6388, Cost Time: 89.00s
Time: 2018-04-06 20:28:47.019060830~2018-04-06 20:43:57.369040719, Loss: 0.7811, Nodes_count: 6398, Cost Time: 89.89s
Time: 2018-04-06 20:43:57.369040719~2018-04-06 20:59:02.029030474, Loss: 1.3277, Nodes_count: 6999, Cost Time: 91.99s
Time: 2018-04-06 20:59:02.029030474~2018-04-06 21:15:00.619010803, Loss: 0.8925, Nodes_count: 7045, Cost Time: 93.04s
Time: 2018-04-06 21:15:00.619010803~2018-04-06 21:30:00.778978760, Loss: 0.5872, Nodes_count: 7061, Cost Time: 94.32s
Time: 2018-04-06 21:30:00.778978760~2018-04-06 21:45:25.218957773, Loss: 0.8202, Nodes_count: 7069, Cost Time: 95.27s
Time: 2018-04-06 21:45:25.218957773~2018-04-06 22:01:32.

In [27]:
model=torch.load("./models/model_saved_share.pt",map_location=device)
memory,gnn, link_pred,neighbor_loader=model
ans_4_7=test_day_new(graph_4_7,"graph_4_7_share")

after merge: TemporalData(dst=[2284034], msg=[2284034, 39], src=[2284034], t=[2284034])
Time: 2018-04-07 00:00:00.008778912~2018-04-07 00:15:00.638758012, Loss: 0.8253, Nodes_count: 251, Cost Time: 0.88s
Time: 2018-04-07 00:15:00.638758012~2018-04-07 00:30:00.678739107, Loss: 1.2567, Nodes_count: 16289, Cost Time: 5.76s
Time: 2018-04-07 00:30:00.678739107~2018-04-07 00:45:48.008718917, Loss: 0.7578, Nodes_count: 16302, Cost Time: 6.82s
Time: 2018-04-07 00:45:48.008718917~2018-04-07 01:01:00.048699594, Loss: 0.7496, Nodes_count: 16315, Cost Time: 7.68s
Time: 2018-04-07 01:01:00.048699594~2018-04-07 01:17:08.118674396, Loss: 0.5200, Nodes_count: 16346, Cost Time: 8.89s
Time: 2018-04-07 01:17:08.118674396~2018-04-07 01:32:15.738656469, Loss: 0.7128, Nodes_count: 16356, Cost Time: 9.71s
Time: 2018-04-07 01:32:15.738656469~2018-04-07 01:48:06.728632339, Loss: 0.7287, Nodes_count: 16367, Cost Time: 10.52s
Time: 2018-04-07 01:48:06.728632339~2018-04-07 02:04:30.508619128, Loss: 0.7368, Nodes_

Time: 2018-04-07 17:43:22.657350334~2018-04-07 17:59:27.637330849, Loss: 1.0929, Nodes_count: 23998, Cost Time: 92.12s
Time: 2018-04-07 17:59:27.637330849~2018-04-07 18:14:38.467310967, Loss: 0.7987, Nodes_count: 24007, Cost Time: 93.10s
Time: 2018-04-07 18:14:38.467310967~2018-04-07 18:29:44.957289774, Loss: 0.7432, Nodes_count: 24027, Cost Time: 94.67s
Time: 2018-04-07 18:29:44.957289774~2018-04-07 18:45:53.477266719, Loss: 1.1435, Nodes_count: 24400, Cost Time: 96.80s
Time: 2018-04-07 18:45:53.477266719~2018-04-07 19:01:01.807254480, Loss: 0.8061, Nodes_count: 24415, Cost Time: 97.84s
Time: 2018-04-07 19:01:01.807254480~2018-04-07 19:16:09.237229592, Loss: 0.5545, Nodes_count: 24428, Cost Time: 99.05s
Time: 2018-04-07 19:16:09.237229592~2018-04-07 19:31:16.917209075, Loss: 0.8908, Nodes_count: 24447, Cost Time: 100.22s
Time: 2018-04-07 19:31:16.917209075~2018-04-07 19:46:24.177188164, Loss: 0.7914, Nodes_count: 24456, Cost Time: 101.10s
Time: 2018-04-07 19:46:24.177188164~2018-04-07

## 计算异常分数函数定义

In [34]:
def cal_train_IDF(find_str,file_list):
    include_count=0
    for f_path in (file_list):
        f=open(f_path)
        if find_str in f.read():
            include_count+=1             
    IDF=math.log(len(file_list)/(include_count+1))
    return IDF


def cal_IDF(find_str,file_path,file_list):
    file_list=os.listdir(file_path)
    include_count=0
    different_neighbor=set()#至少有两个 因此要减去2
    for f_path in (file_list):
        f=open(file_path+f_path)
        if find_str in f.read():
            include_count+=1                
                
    IDF=math.log(len(file_list)/(include_count+1))
    
    return IDF,1

def cal_redundant(find_str,edge_list):
    
    different_neighbor=set()#至少有两个 因此要减去2
    for e in edge_list:
        if find_str in str(e):
            different_neighbor.add(e[0])
            different_neighbor.add(e[1])
    return len(different_neighbor)-2

def cal_anomaly_loss(loss_list,edge_list,file_path):
    
    if len(loss_list)!=len(edge_list):
        print("error!")
        return 0
    count=0
    loss_sum=0
    loss_std=std(loss_list)
    loss_mean=mean(loss_list)
    edge_set=set()
    node_set=set()
    node2redundant={}
    
    thr=loss_mean+1.5*loss_std

    print("thr:",thr)

    for i in range(len(loss_list)):
        if loss_list[i]>thr:
            count+=1
            src_node=edge_list[i][0]
            dst_node=edge_list[i][1]
            
            loss_sum+=loss_list[i]
    
            node_set.add(src_node)
            node_set.add(dst_node)
            edge_set.add(edge_list[i][0]+edge_list[i][1])
    return count, loss_sum/count,node_set,edge_set
#     return count, count/len(loss_list)

In [35]:
# 进行计算节点的IDF分数字典
node_IDF={}
node_set=set()

file_list=[]

# file_path="graph_4_2_share//"
# file_l=os.listdir("graph_4_2_share//")
# for i in file_l:
#     file_list.append(file_path+i)

file_path="graph_4_3_share//"
file_l=os.listdir("graph_4_3_share//")
for i in file_l:
    file_list.append(file_path+i)

file_path="graph_4_4_share//"
file_l=os.listdir("graph_4_4_share//")
for i in file_l:
    file_list.append(file_path+i)

file_path="graph_4_5_share/"
file_l=os.listdir("graph_4_5_share//")
for i in file_l:
    file_list.append(file_path+i)
    

# Original codes
# for f_path in tqdm(file_list):
#     f=open(f_path)
#     for line in f:
#         l=line.strip()
#         jdata=eval(l)
#         if jdata['loss']>0:            
#             if 'netflow' not in str(jdata['srcmsg']):
#                 node_set.add(str(jdata['srcmsg']))
#             if 'netflow' not in str(jdata['dstmsg']):
#                 node_set.add(str(jdata['dstmsg'])) 

# for n in tqdm(node_set):
#     find_str=list(eval(n).values())[0]
#     IDF=cal_train_IDF(n,file_list)
#     node_IDF[n]=IDF
    
    
# The optimized codes  
node_set = {}
for f_path in tqdm(file_list):
    f=open(f_path)
    for line in f:
        l=line.strip()
        jdata=eval(l)
        jdata=eval(l)
        if jdata['loss']>0:
            if 'netflow' not in str(jdata['srcmsg']):
                if str(jdata['srcmsg']) not in node_set.keys():
                    node_set[str(jdata['srcmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['srcmsg'])].add(f_path)
            if 'netflow' not in str(jdata['dstmsg']):
                if str(jdata['dstmsg']) not in node_set.keys():
                    node_set[str(jdata['dstmsg'])] = set([f_path])
                else:
                    node_set[str(jdata['dstmsg'])].add(f_path)
for n in node_set:
    include_count = len(node_set[n])   
    IDF=math.log(len(file_list)/(include_count+1))
    node_IDF[n] = IDF     
    
  
    
#     if 'subject' in n:
#         node_IDF[n]=IDF*1
#     else:
#         node_IDF[n]=IDF*diff_ner

torch.save(node_IDF,"node_IDF_4_3-4_5")
print("IDF weight calculate complete!")

100%|████████████████████████████████████████████████████████████████████████████████████████| 277/277 [01:42<00:00,  2.70it/s]

IDF weight calculate complete!


## 窗口关系建立函数

In [37]:
file_list=[]

file_path="graph_4_3_share//"
file_l=os.listdir("graph_4_3_share//")
for i in file_l:
    file_list.append(file_path+i)

file_path="graph_4_4_share//"
file_l=os.listdir("graph_4_4_share//")
for i in file_l:
    file_list.append(file_path+i)

file_path="graph_4_5_share//"
file_l=os.listdir("graph_4_5_share//")
for i in file_l:
    file_list.append(file_path+i)
    
# 下面这个函数，开源时加comment，为了剔除common nodes
def is_include_key_word(s):
    keywords=[
         'netflow',
        '/home/george/Drafts' ,
        'usr',
        'proc',
        'var',
        'cadet',
        '/var/log/debug.log',
        '/var/log/cron',
        '/home/charles/Drafts',
        '/etc/ssl/cert.pem',
        '/tmp/.31.3022e',  
      ]
    flag=False
    for i in keywords:
        if i in s:
            flag=True
    return flag
    
def cal_set_rel(s1,s2):
    new_s=s1 & s2
    count=0
#     node_IDF=torch.load("node_IDF_4_3-4_5")
    for i in new_s:
        
# 下面这个函数，开源时加comment，为了剔除common nodes
        if is_include_key_word(i) is True:
            node_IDF[i]=math.log(len(file_list)/(1+len(file_list)))
            
        if i in node_IDF.keys():
            IDF=node_IDF[i]
        else:
            IDF=math.log(len(file_list)/(1))
#             print(IDF)
        if IDF>(math.log(len(file_list)*0.9)):
            print("node:",i," IDF:",IDF)
            count+=1
    return count

In [44]:
# 通过4-5的数据来确认阈值
import copy
# node_IDF=torch.load("node_IDF_4_3-4_5")
y_data_4_5=[]
df_list_4_5=[]
# node_set_list=[]
history_list=[]
tw_que=[]
his_tw={}
current_tw={}

file_l=os.listdir("graph_4_5_share/")
index_count=0
for f_path in sorted(file_l):
    f=open("graph_4_5_share/"+f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_4_5.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_4_5_share//")
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'])!=0 and current_tw['name']!=his_tw['name']:
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list.append(temp_hq)
  
    index_count+=1
#     node_set_list.append(node_set)
    print( f_path,"  ",loss_avg," num of anomalous edges within the time window:",count," percentage of anomalous edges:",count/len(edge_list)," anomalous node count:",len(node_set)," anomalous edge count:",len(edge_set))
    #     y_data_4_10.append([loss_avg,labels_4_10[f_path],f_path])

index_count: 0
thr: 2.0254392510743475
2018-04-05 00:00:00.019021794~2018-04-05 00:15:00.828996051.txt    2.4401597110687923  num of anomalous edges within the time window: 1327  percentage of anomalous edges: 0.10799153645833333  anomalous node count: 74  anomalous edge count: 93
index_count: 1
thr: 1.5968371127055037
2018-04-05 00:15:00.828996051~2018-04-05 00:31:53.958984540.txt    2.126877556964881  num of anomalous edges within the time window: 2372  percentage of anomalous edges: 0.1286892361111111  anomalous node count: 82  anomalous edge count: 116
index_count: 2
thr: 1.9736710775905066
2018-04-05 00:31:53.958984540~2018-04-05 00:47:02.738955442.txt    2.390264725988329  num of anomalous edges within the time window: 1494  percentage of anomalous edges: 0.12158203125  anomalous node count: 54  anomalous edge count: 71
index_count: 3
thr: 1.9395233924079043
2018-04-05 00:47:02.738955442~2018-04-05 01:02:10.248933422.txt    2.362452552847992  num of anomalous edges within the tim

thr: 1.9982512125470158
2018-04-05 07:59:37.718371772~2018-04-05 08:15:00.048354234.txt    2.4055915902779756  num of anomalous edges within the time window: 1432  percentage of anomalous edges: 0.11653645833333333  anomalous node count: 61  anomalous edge count: 81
index_count: 32
thr: 1.5606624812397631
2018-04-05 08:15:00.048354234~2018-04-05 08:30:00.818334493.txt    2.1269149841066306  num of anomalous edges within the time window: 2157  percentage of anomalous edges: 0.12390854779411764  anomalous node count: 85  anomalous edge count: 124
index_count: 33
thr: 1.9685890198492606
2018-04-05 08:30:00.818334493~2018-04-05 08:46:58.358308872.txt    2.38025044403535  num of anomalous edges within the time window: 1455  percentage of anomalous edges: 0.118408203125  anomalous node count: 54  anomalous edge count: 70
index_count: 34
thr: 2.009741987120438
2018-04-05 08:46:58.358308872~2018-04-05 09:02:14.208291024.txt    2.384720639114768  num of anomalous edges within the time window: 1

thr: 2.6680127821692197
2018-04-05 15:41:32.747754581~2018-04-05 15:57:39.747734089.txt    3.7653659456409514  num of anomalous edges within the time window: 1536  percentage of anomalous edges: 0.06818181818181818  anomalous node count: 132  anomalous edge count: 154
index_count: 62
thr: 2.4945387120850366
2018-04-05 15:57:39.747734089~2018-04-05 16:13:34.727713638.txt    3.2513579003175193  num of anomalous edges within the time window: 1699  percentage of anomalous edges: 0.07900855654761904  anomalous node count: 84  anomalous edge count: 111
index_count: 63
thr: 2.4922714208870484
2018-04-05 16:13:34.727713638~2018-04-05 16:29:02.837699924.txt    3.536134474732341  num of anomalous edges within the time window: 2385  percentage of anomalous edges: 0.08031384698275862  anomalous node count: 114  anomalous edge count: 154
index_count: 64
thr: 2.1712520618797724
2018-04-05 16:29:02.837699924~2018-04-05 16:44:03.257674146.txt    2.7024503208097372  num of anomalous edges within the ti

thr: 1.8017819048409027
2018-04-05 23:30:00.957128263~2018-04-05 23:45:01.057108814.txt    2.0596920655361854  num of anomalous edges within the time window: 13049  percentage of anomalous edges: 0.11277136338495575  anomalous node count: 6301  anomalous edge count: 6415


In [45]:
pred_label={}
anomalous_queue_scores = []

files = os.listdir("graph_4_5_share/")
for f in files:
    pred_label[f]=0


for hl in history_list:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
#     if loss_count>100:
    name_list=[]
    for i in hl:
        name_list.append(i['name'])
    print(name_list)
    for i in name_list:
        pred_label[i]=1
    print(loss_count)
    
    anomalous_queue_scores.append(loss_count)

['2018-04-05 00:00:00.019021794~2018-04-05 00:15:00.828996051.txt']
3.4401597110687923
['2018-04-05 00:15:00.828996051~2018-04-05 00:31:53.958984540.txt']
3.126877556964881
['2018-04-05 00:31:53.958984540~2018-04-05 00:47:02.738955442.txt']
3.390264725988329
['2018-04-05 00:47:02.738955442~2018-04-05 01:02:10.248933422.txt']
3.362452552847992
['2018-04-05 01:02:10.248933422~2018-04-05 01:17:16.198913636.txt']
3.3938867459276416
['2018-04-05 01:17:16.198913636~2018-04-05 01:32:24.868893768.txt']
3.1002497520695957
['2018-04-05 01:32:24.868893768~2018-04-05 01:48:31.268873586.txt']
3.394860729976778
['2018-04-05 01:48:31.268873586~2018-04-05 02:03:38.918851533.txt']
3.355600359117178
['2018-04-05 02:03:38.918851533~2018-04-05 02:19:27.878832963.txt']
3.3382968652733993
['2018-04-05 02:19:27.878832963~2018-04-05 02:34:55.308818790.txt']
3.1457103994984266
['2018-04-05 02:34:55.308818790~2018-04-05 02:51:02.788787835.txt']
3.390060067667056
['2018-04-05 02:51:02.788787835~2018-04-05 03:06:

In [46]:
beta_threshold = max(anomalous_queue_scores)
beta_threshold

5.242535307112918

## 新检测机制 4-6

In [38]:

# node_IDF=torch.load("node_IDF_4_3-4_5")
y_data_4_6=[]
df_list_4_6=[]
# node_set_list=[]
history_list=[]
tw_que=[]
his_tw={}
current_tw={}

file_l=os.listdir("graph_4_6_share/")
index_count=0
for f_path in sorted(file_l):
    f=open("graph_4_6_share/"+f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_4_6.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_4_6_share/")
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'])!=0 and current_tw['name']!=his_tw['name']:
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list.append(temp_hq)
    index_count+=1
    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))


index_count: 0
thr: 1.9908172406211657
2018-04-06 00:00:00.017083676~2018-04-06 00:15:50.177068871.txt    2.4577024042755737  count: 1197  percentage: 0.08349609375  node count: 96  edge count: 154
index_count: 1
thr: 1.6907408764837044
2018-04-06 00:15:50.177068871~2018-04-06 00:30:58.967045191.txt    2.1575900126233303  count: 2679  percentage: 0.130810546875  node count: 93  edge count: 197
index_count: 2
thr: 1.9919519756826864
2018-04-06 00:30:58.967045191~2018-04-06 00:47:07.167023337.txt    2.5226839439954585  count: 1336  percentage: 0.08697916666666666  node count: 78  edge count: 135
index_count: 3
thr: 1.9612414399811182
2018-04-06 00:47:07.167023337~2018-04-06 01:02:14.677010991.txt    2.4173589651466263  count: 1410  percentage: 0.09835379464285714  node count: 89  edge count: 165
index_count: 4
thr: 1.9513866290614739
2018-04-06 01:02:14.677010991~2018-04-06 01:17:19.056982719.txt    2.398199404601927  count: 1232  percentage: 0.10026041666666667  node count: 77  edge cou

thr: 3.0103301936748013
2018-04-06 10:33:00.136241644~2018-04-06 10:48:11.796216358.txt    4.407358262547601  count: 1646  percentage: 0.07654389880952381  node count: 105  edge count: 106
index_count: 42
thr: 2.9526448575217303
2018-04-06 10:48:11.796216358~2018-04-06 11:03:19.756210028.txt    3.994000849696253  count: 2076  percentage: 0.08814538043478261  node count: 63  edge count: 64
index_count: 43
thr: 2.9980328642466674
2018-04-06 11:03:19.756210028~2018-04-06 11:18:26.126177915.txt    3.6808227307550156  count: 6214  percentage: 0.1213671875  node count: 558  edge count: 559
index_count: 44
thr: 1.907028797497779
node: {'file': '/home/user/.links/links.his'}  IDF: 5.6240175061873385
node: {'subject': 'links'}  IDF: 5.6240175061873385
2018-04-06 11:18:26.126177915~2018-04-06 11:33:35.116170745.txt    2.541123864357893  count: 2182  percentage: 0.09685724431818182  node count: 138  edge count: 243
index_count: 45
thr: 2.1859289705376264
node: {'subject': 'vUgefal'}  IDF: 5.62401

thr: 2.5848167496047556
2018-04-06 22:32:48.168897833~2018-04-06 22:48:06.838874079.txt    3.761243191615698  count: 1301  percentage: 0.07473575367647059  node count: 66  edge count: 77
index_count: 82
thr: 2.264000268496172
2018-04-06 22:48:06.838874079~2018-04-06 23:03:09.188854937.txt    3.47918757774847  count: 1946  percentage: 0.02500513980263158  node count: 171  edge count: 195
index_count: 83
thr: 2.3139839100845307
node: {'file': '/tmp/periodic.L7oz7KTJ1r'}  IDF: 5.6240175061873385
2018-04-06 23:03:09.188854937~2018-04-06 23:18:10.168840981.txt    2.977040430157053  count: 6193  percentage: 0.05399867466517857  node count: 1216  edge count: 1226
index_count: 84
thr: 1.9532320057063668
2018-04-06 23:18:10.168840981~2018-04-06 23:33:15.408814807.txt    2.3611681694697997  count: 1415  percentage: 0.10629507211538461  node count: 77  edge count: 142
index_count: 85
thr: 1.7305920435591648
node: {'file': '/tmp/periodic.L7oz7KTJ1r'}  IDF: 5.6240175061873385
2018-04-06 23:33:15.40

In [40]:

pred_label={}

filelist = os.listdir("graph_4_6_share")
for f in filelist:
    pred_label[f]=0
    
filelist = os.listdir("graph_4_7_share")
for f in filelist:
    pred_label[f]=0

for hl in history_list:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
    if loss_count>100:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
        print(name_list)
        for i in name_list:
            pred_label[i]=1
        print(loss_count)

['2018-04-06 11:03:19.756210028~2018-04-06 11:18:26.126177915.txt', '2018-04-06 11:18:26.126177915~2018-04-06 11:33:35.116170745.txt', '2018-04-06 11:33:35.116170745~2018-04-06 11:48:42.606135188.txt', '2018-04-06 11:48:42.606135188~2018-04-06 12:03:50.186115455.txt', '2018-04-06 12:03:50.186115455~2018-04-06 14:01:32.489584227.txt']
918.1379802082796


## 新检测机制 4-7

In [41]:

# node_IDF=torch.load("node_IDF_4_3-4_5")
y_data_4_7=[]
df_list_4_7=[]
# node_set_list=[]
history_list=[]
tw_que=[]
his_tw={}
current_tw={}

file_l=os.listdir("graph_4_7_share")
index_count=0
for f_path in sorted(file_l):
    f=open("graph_4_7_share/"+f_path)
    edge_loss_list=[]
    edge_list=[]
    print('index_count:',index_count)
    
    for line in f:
        l=line.strip()
        jdata=eval(l)
        edge_loss_list.append(jdata['loss'])
        edge_list.append([str(jdata['srcmsg']),str(jdata['dstmsg'])])
    df_list_4_7.append(pd.DataFrame(edge_loss_list))
    count,loss_avg,node_set,edge_set=cal_anomaly_loss(edge_loss_list,edge_list,"graph_4_7_share/")
    current_tw['name']=f_path
    current_tw['loss']=loss_avg
    current_tw['index']=index_count
    current_tw['nodeset']=node_set

    added_que_flag=False
    for hq in history_list:
        for his_tw in hq:
            if cal_set_rel(current_tw['nodeset'],his_tw['nodeset'])!=0 and current_tw['name']!=his_tw['name']:
                hq.append(copy.deepcopy(current_tw))
                added_que_flag=True
                break
            if added_que_flag:
                break
    if added_que_flag is False:
        temp_hq=[copy.deepcopy(current_tw)]
        history_list.append(temp_hq)
  
    index_count+=1

    print( f_path,"  ",loss_avg," count:",count," percentage:",count/len(edge_list)," node count:",len(node_set)," edge count:",len(edge_set))


index_count: 0
thr: 2.0475062222283578
2018-04-07 00:00:00.008778912~2018-04-07 00:15:00.638758012.txt    2.5784202080257135  count: 1103  percentage: 0.07180989583333333  node count: 131  edge count: 160
index_count: 1
thr: 2.2583148570981004
node: {'file': '/tmp/periodic.TWkmZ7gwrS'}  IDF: 5.6240175061873385
2018-04-07 00:15:00.638758012~2018-04-07 00:30:00.678739107.txt    3.338940780994126  count: 2346  percentage: 0.029753449675324676  node count: 392  edge count: 408
index_count: 2
thr: 2.0111376011754905
2018-04-07 00:30:00.678739107~2018-04-07 00:45:48.008718917.txt    2.478785437202601  count: 1293  percentage: 0.09019252232142858  node count: 75  edge count: 129
index_count: 3
thr: 1.983675478274942
2018-04-07 00:45:48.008718917~2018-04-07 01:01:00.048699594.txt    2.4302555864991664  count: 1343  percentage: 0.09368024553571429  node count: 87  edge count: 153
index_count: 4
thr: 1.6608844401213747
2018-04-07 01:01:00.048699594~2018-04-07 01:17:08.118674396.txt    2.09869124

thr: 3.1442799836437
2018-04-07 10:51:55.367905007~2018-04-07 11:07:01.987882382.txt    4.989996227512224  count: 1547  percentage: 0.0604296875  node count: 74  edge count: 72
index_count: 43
thr: 2.7745876959090037
node: {'file': '/home/user/#pico21309#'}  IDF: 5.6240175061873385
2018-04-07 11:07:01.987882382~2018-04-07 11:22:08.247863262.txt    3.866933889565091  count: 2871  percentage: 0.05965342420212766  node count: 225  edge count: 240
index_count: 44
thr: 2.583160968871651
2018-04-07 11:22:08.247863262~2018-04-07 11:37:19.117845222.txt    4.117893970669057  count: 1126  percentage: 0.061089409722222224  node count: 72  edge count: 85
index_count: 45
thr: 2.9865511635119364
2018-04-07 11:37:19.117845222~2018-04-07 11:52:25.757824807.txt    4.473294316008028  count: 1808  percentage: 0.06790865384615384  node count: 118  edge count: 127
index_count: 46
thr: 2.356769615449112
2018-04-07 11:52:25.757824807~2018-04-07 12:07:33.167805575.txt    3.412803858266276  count: 1063  percen

thr: 2.283745204447551
2018-04-07 21:18:08.817071709~2018-04-07 21:33:17.447051923.txt    3.4173430458028267  count: 940  percentage: 0.061197916666666664  node count: 64  edge count: 83
index_count: 84
thr: 2.3969516117480847
2018-04-07 21:33:17.447051923~2018-04-07 21:48:25.107022873.txt    3.9173935518277334  count: 759  percentage: 0.052943638392857144  node count: 62  edge count: 78
index_count: 85
thr: 2.013769330180591
2018-04-07 21:48:25.107022873~2018-04-07 22:03:32.547001547.txt    2.4648629502473374  count: 1326  percentage: 0.09249441964285714  node count: 94  edge count: 147
index_count: 86
thr: 2.0459545678709796
2018-04-07 22:03:32.547001547~2018-04-07 22:18:38.656982046.txt    2.9150453110196892  count: 1758  percentage: 0.08175223214285714  node count: 106  edge count: 165
index_count: 87
thr: 2.7640029955787058
2018-04-07 22:18:38.656982046~2018-04-07 22:33:47.286964532.txt    4.227548967002744  count: 1106  percentage: 0.06353400735294118  node count: 65  edge count:

In [42]:

for hl in history_list:
    loss_count=0
    for hq in hl:
        if loss_count==0:
            loss_count=(loss_count+1)*(hq['loss']+1)
        else:
            loss_count=(loss_count)*(hq['loss']+1)
    name_list=[]
    if loss_count>100:
        name_list=[]
        for i in hl:
            name_list.append(i['name'])
        print(name_list)
        # 下面先打上注释意味着先看下测试结果，先不往预测结果中加
#         for i in name_list:
#             pred_label[i]=1
        print(loss_count)

# 异常检测模块

In [43]:
from sklearn.metrics import average_precision_score, roc_auc_score
#导入plot_roc_curve,roc_curve和roc_auc_score模块
# from sklearn.metrics import plot_roc_curve,roc_curve,auc,roc_auc_score
import torch
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

def plot_thr():
    np.seterr(invalid='ignore')
    step=0.01
    thr_list=torch.arange(-5,5,step)
    
    

    precision_list=[]
    recall_list=[]
    fscore_list=[]
    accuracy_list=[]
    auc_val_list=[]
    for thr in thr_list:
        threshold=thr
        y_prediction=[]
        for i in y_test_scores:
            if i >threshold:
                y_prediction.append(1)
            else:
                y_prediction.append(0)
        precision,recall,fscore,accuracy,auc_val=classifier_evaluation(y_test, y_prediction)   
        precision_list.append(float(precision))
        recall_list.append(float(recall))
        fscore_list.append(float(fscore))
        accuracy_list.append(float(accuracy))
        auc_val_list.append(float(auc_val))

    max_fscore=max(fscore_list)
    max_fscore_index=fscore_list.index(max_fscore)
    print(max_fscore_index)
    print("max threshold:",thr_list[max_fscore_index])
    print('precision:',precision_list[max_fscore_index])
    print('recall:',recall_list[max_fscore_index])
    print('fscore:',fscore_list[max_fscore_index])
    print('accuracy:',accuracy_list[max_fscore_index])    
    print('auc:',auc_val_list[max_fscore_index])

    plt.plot(thr_list,precision_list,color='red',label='precision',linewidth=2.0,linestyle='-')
    plt.plot(thr_list,recall_list,color='orange',label='recall',linewidth=2.0,linestyle='solid')
    plt.plot(thr_list,fscore_list,color='y',label='F-score',linewidth=2.0,linestyle='dashed')
    plt.plot(thr_list,accuracy_list,color='g',label='accuracy',linewidth=2.0,linestyle='dashdot')
    plt.plot(thr_list,auc_val_list,color='b',label='auc_val',linewidth=2.0,linestyle='dotted')


    plt.xlabel("Threshold", fontdict={'size': 16})
    plt.ylabel("Rate", fontdict={'size': 16})
    plt.title("Different evaluation Indicators by varying threshold value", fontdict={'size': 12})
    plt.legend(loc='best', fontsize=12, markerscale=0.5)
    plt.show()

def classifier_evaluation(y_test, y_test_pred):
    # groundtruth, pred_value
    tn, fp, fn, tp =confusion_matrix(y_test, y_test_pred).ravel()
#     tn+=100
#     print(clf_name," : ")
    print('tn:',tn)
    print('fp:',fp)
    print('fn:',fn)
    print('tp:',tp)
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    accuracy=(tp+tn)/(tp+tn+fp+fn)
    fscore=2*(precision*recall)/(precision+recall)    
    auc_val=roc_auc_score(y_test, y_test_pred)
    print("precision:",precision)
    print("recall:",recall)
    print("fscore:",fscore)
    print("accuracy:",accuracy)
    print("auc_val:",auc_val)
    return precision,recall,fscore,accuracy,auc_val

def minmax(data):
    min_val=min(data)
    max_val=max(data)
    ans=[]
    for i in data:
        ans.append((i-min_val)/(max_val-min_val))
    return ans


In [44]:
y=[]
y_pred=[]
for i in labels:
    y.append(labels[i])
    y_pred.append(pred_label[i])

In [45]:
classifier_evaluation(y,y_pred)

tn: 174
fp: 1
fn: 0
tp: 4
precision: 0.8
recall: 1.0
fscore: 0.888888888888889
accuracy: 0.994413407821229
auc_val: 0.9971428571428572


(0.8, 1.0, 0.888888888888889, 0.994413407821229, 0.9971428571428572)

# Attack investigation

In [31]:
from graphviz import Digraph
import networkx as nx
# 社区发现算法
import datetime
import community.community_louvain as community_louvain
from tqdm import tqdm

In [42]:

# 攻击调查模块

# 一些常见的路径替换  需要根据不同数据集进行调整
replace_dic={
    '/run/shm/':'/run/shm/*',
    '/home/admin/.cache/mozilla/firefox/':'/home/admin/.cache/mozilla/firefox/*',
    '/home/admin/.mozilla/firefox':'/home/admin/.mozilla/firefox*',    
    '/data/replay_logdb/':'/data/replay_logdb/*', 
    '/home/admin/.local/share/applications/':'/home/admin/.local/share/applications/*', 
    '/usr/share/applications/':'/usr/share/applications/*', 
    '/lib/x86_64-linux-gnu/':'/lib/x86_64-linux-gnu/*',     
    '/proc/':'/proc/*', 
    '/stat':'*/stat', 
    '/etc/bash_completion.d/':'/etc/bash_completion.d/*', 
    '/usr/bin/python2.7':'/usr/bin/python2.7/*', 
    '/usr/lib/python2.7':'/usr/lib/python2.7/*', 
}

def replace_path_name(path_name):
    for i in replace_dic:
        if i in path_name:
            return replace_dic[i]
    return path_name


# 异常检测发现的时间窗口
attack_list=[
    # 这里存放有问题的时间窗口的edge list
    './1024_50_genIDF/graph_4_6_share/2018-04-06 11:18:26.126177915~2018-04-06 11:33:35.116170745.txt',
    './1024_50_genIDF/graph_4_6_share/2018-04-06 11:33:35.116170745~2018-04-06 11:48:42.606135188.txt',
    './1024_50_genIDF/graph_4_6_share/2018-04-06 11:48:42.606135188~2018-04-06 12:03:50.186115455.txt',
    './1024_50_genIDF/graph_4_6_share/2018-04-06 12:03:50.186115455~2018-04-06 14:01:32.489584227.txt', 
  ]

# 生成加权图
original_edges_count=0
graphs=[]
gg=nx.DiGraph()
count=0
for path in tqdm(attack_list):
    if ".txt" in path:
        line_count=0
        node_set=set()
        tempg=nx.DiGraph()
        f=open(path,"r")       
        edge_list=[]
        for line in f:
            count+=1
            l=line.strip()
            jdata=eval(l)
            edge_list.append(jdata)
            
        edge_list = sorted(edge_list, key=lambda x:x['loss'],reverse=True) 
        original_edges_count+=len(edge_list)
        
        loss_list=[]
        for i in edge_list:
            loss_list.append(i['loss'])
        loss_mean=mean(loss_list)
        loss_std=std(loss_list)
        print(loss_mean)
        print(loss_std)
        thr=loss_mean+1.5*loss_std
        print("thr:",thr)
        for e in edge_list:
            if e['loss']>thr:    
                tempg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))))
                gg.add_edge(str(hashgen(replace_path_name(e['srcmsg']))),str(hashgen(replace_path_name(e['dstmsg']))),loss=e['loss'],srcmsg=e['srcmsg'],dstmsg=e['dstmsg'],edge_type=e['edge_type'],time=e['time'])
        


# starttime = datetime.datetime.now()
#long running
partition = community_louvain.best_partition(gg.to_undirected())
#do something other
# endtime = datetime.datetime.now()
# print("社区发现计算完毕，消耗时间:{:d}".format((endtime - starttime).seconds))

# 根据社区构建candidate subgraph
communities={}
max_partition=0
for i in partition:
    if partition[i]>max_partition:
        max_partition=partition[i]
for i in range(max_partition+1):
    communities[i]=nx.DiGraph()
for e in gg.edges:
#     if partition[e[0]]==partition[e[1]]:
    communities[partition[e[0]]].add_edge(e[0],e[1])
    communities[partition[e[1]]].add_edge(e[0],e[1])


# 定义attack node，将对应的edge和node在绘制时染色
def attack_edge_flag(msg):
    attack_edge_type=[
        '/tmp/vUgefal',
        'vUgefal',
        '/var/log/devc',
        '/etc/passwd',
        '81.49.200.166',
        '61.167.39.128',
        '78.205.235.65',
        '139.123.0.113',
        "'nginx'",
    ]
    flag=False
    for i in attack_edge_type:
        if i in msg:
            flag=True
    return flag


# 绘制和渲染candidate subgraph




graph_index=0


for c in communities:
    # 将文件、 进程、 网络连接的信息放到节点上面去
    dot = Digraph(name="MyPicture", comment="the test", format="pdf")
    dot.graph_attr['rankdir'] = 'LR'
    
    for e in communities[c].edges:
        try:
            temp_edge=gg.edges[e]
            srcnode=e['srcnode']
            dstnode=e['dstnode']
        except:
            pass        

        if True:
#             subgraph_loss_sum+=temp_edge['loss']
            # 源节点     
            if "'subject': '" in temp_edge['srcmsg']:
                src_shape='box'
            elif "'file': '" in temp_edge['srcmsg']:
                src_shape='oval'
            elif "'netflow': '" in temp_edge['srcmsg']:
                src_shape='diamond'
            if attack_edge_flag(temp_edge['srcmsg']):
                src_node_color='red'
#                 attack_node_count+=1
            else:
                src_node_color='blue'
            dot.node( name=str(hashgen(replace_path_name(temp_edge['srcmsg']))),label=str(replace_path_name(temp_edge['srcmsg'])+str(partition[str(hashgen(replace_path_name(temp_edge['srcmsg'])))])), color=src_node_color,shape = src_shape)

            #目的节点
            if "'subject': '" in temp_edge['dstmsg']:
                dst_shape='box'
            elif "'file': '" in temp_edge['dstmsg']:
                dst_shape='oval'
            elif "'netflow': '" in temp_edge['dstmsg']:
                dst_shape='diamond'
            if attack_edge_flag(temp_edge['dstmsg']):
                dst_node_color='red'
#                 attack_node_count+=1
            else:
                dst_node_color='blue'
            dot.node( name=str(hashgen(replace_path_name(temp_edge['dstmsg']))),label=str(replace_path_name(temp_edge['dstmsg'])+str(partition[str(hashgen(replace_path_name(temp_edge['dstmsg'])))])), color=dst_node_color,shape = dst_shape)

            if attack_edge_flag(temp_edge['srcmsg']) and attack_edge_flag(temp_edge['dstmsg']):
                edge_color='red'
#                 attack_edge_count+=1
            else:
                edge_color='blue'
            dot.edge(str(hashgen(replace_path_name(temp_edge['srcmsg']))),str(hashgen(replace_path_name(temp_edge['dstmsg']))), label= temp_edge['edge_type'] , color=edge_color)#+ "  loss: "+str(temp_edge['loss']) + "  time: "+str(temp_edge['time'])

    print("开始渲染图像···")
    dot.render('./graph_visual/subgraph_'+str(graph_index), view=False)
    graph_index+=1


 50%|██████████████████████████████████████████████                                              | 2/4 [00:00<00:00,  5.43it/s]

0.6261387372864688
0.8539267068075401
thr: 1.907028797497779
0.8300467786579248
0.9039214612531344
thr: 2.1859289705376264


100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  6.17it/s]

0.9493427947505148
0.9534912391819826
thr: 2.3795796535234888
0.8755370802948619
0.8392828735783188
thr: 2.1344613906623398
开始渲染图像···
开始渲染图像···
开始渲染图像···
开始渲染图像···
开始渲染图像···


开始渲染图像···
开始渲染图像···
开始渲染图像···
开始渲染图像···
开始渲染图像···
开始渲染图像···
开始渲染图像···
开始渲染图像···
开始渲染图像···


In [39]:
def attack_edge_flag(msg):
    attack_edge_type=[
        '/home/admin/clean',
        '/dev/glx_alsa_675',
        '/home/admin/profile',
          '/var/log/xdev',
    '/etc/passwd',
    '161.116.88.72',
    '146.153.68.151',
        '/var/log/mail',
        '/tmp/memtrace.so',
        '/var/log/xdev',
         '/var/log/wdev',
        'gtcache',
        'firefox',
#         '/var/log',
        'vUgefal'
    ]
    flag=False
    for i in attack_edge_type:
        if i in msg:
            flag=True
            break
    return flag


In [40]:
attack_edge_flag("{'subject': 'vUgefal'}")

True